## Import Library

In [1]:
import pandas as pd
import numpy as np 
import sklearn
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, accuracy_score

## Data Preprocessing

In [2]:
df_Stock = pd.read_csv("Desktop/finalproject_training.csv")
comp_list = df_Stock.comp_id.unique()

### Create next month return

df_Stock["m_next_ret"] = df_Stock.groupby('comp_id')["m_ret"].shift(-1)
df_Stock = df_Stock.dropna(subset = ["m_next_ret"])

### Construct self-defined feature

df_Stock["h/l"] = df_Stock["m_high_adj"] / df_Stock["m_low_adj"] - 1
df_Stock["d/p"] = df_Stock["m_divs"] / df_Stock["close_adj"]
df_Stock["log_m_volume_adj"] = np.log(df_Stock["m_volume_adj"] + 1)
df_Stock["log_SP500"] = np.log(df_Stock["SP500WeeklyClose"] + 1)


feature_names = ["m_ret", "d/p", "h/l", "log_m_volume_adj", "log_SP500", 
                 "Bullish", "Bearish", "Bullish8WeekMovAvg", 
                 "epsfxq",  "mkvaltq", "gsector"]

### Select company based on data availability of feature
comp_selection = comp_list
for feature in feature_names:
    comp_selection = comp_selection[df_Stock.groupby(df_Stock.comp_id, sort = False)[feature]\
        .apply(lambda x: not x.isnull().all())]
    df_Stock = df_Stock[df_Stock.comp_id.isin(comp_selection)]
    df_Stock[feature] = df_Stock.groupby(df_Stock.comp_id, sort = False)[feature].apply(lambda x: x.ffill().bfill()) 

### Create Dummy Variables for feature gsector

non_dummy_cols = df_Stock.columns
df_Stock = pd.get_dummies(df_Stock, columns = ["gsector"])
dummy_cols = list(set(df_Stock.columns) - set(non_dummy_cols))

feature_names = ["m_ret", "d/p", "h/l", "log_m_volume_adj", "log_SP500", 
                 "Bullish", "Bearish", "Bullish8WeekMovAvg", 
                 "epsfxq",  "mkvaltq"] + dummy_cols

### Construct training dataset

X_ret = df_Stock.loc[:, feature_names]
y_ret = df_Stock.loc[:, "m_next_ret"]

X_gof = df_Stock.loc[:, feature_names]
y_gof = ((1 + np.sign(y_ret)) / 2).astype("int")

## Construct an iterable of training-test splits for Cross Validation via ShufflieSplit

In [3]:
from sklearn.model_selection import ShuffleSplit

cross_validation_shufflesplit = ShuffleSplit(n_splits = 10, test_size = 0.25)

## Random Forest Regression

In [4]:
from sklearn.ensemble import RandomForestRegressor

RandomForest_regressor = RandomForestRegressor()
RandomForest_regressor_pipeline = make_pipeline(StandardScaler(), RandomForest_regressor)
RandomForest_regressor_param_grid = dict(randomforestregressor__n_estimators=[5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
                                         randomforestregressor__max_features = [None, "sqrt", "log2"])

RandomForest_regressor_grid_search = GridSearchCV(estimator = RandomForest_regressor_pipeline, 
                                                  param_grid=RandomForest_regressor_param_grid, 
                                                  cv = cross_validation_shufflesplit, 
                                                  n_jobs = -1,
                                                  scoring = "neg_mean_squared_error")
RandomForest_regressor_grid_search.fit(X_ret, y_ret)

means = RandomForest_regressor_grid_search.cv_results_['mean_test_score']
stds = RandomForest_regressor_grid_search.cv_results_['std_test_score']

print("Mean Test Score:", means)
print("Standard Deviation of Test Score:", stds)
print('Optimal Model Parameters: ' + str(RandomForest_regressor_grid_search.best_params_))

### Serialize the gridsearchcv result

# joblib.dump(RandomForest_regressor_grid_search, "Serialized_Predictor\\RandomForest_regressor_grid_search.serialized")

Mean Test Score: [-0.61380632 -0.56376112 -0.5599388  -0.5441412  -0.54534636 -0.52877587
 -0.53780405 -0.53219729 -0.5333321  -0.5369731  -0.54880506 -0.50581117
 -0.50844144 -0.49239419 -0.48444644 -0.4806274  -0.47674135 -0.47696241
 -0.47274604 -0.47054682 -0.54291205 -0.49542957 -0.50034279 -0.48614758
 -0.4896322  -0.4829976  -0.47795438 -0.47156437 -0.47388733 -0.47677498]
Standard Deviation of Test Score: [0.23034613 0.21241416 0.20433146 0.20043095 0.20839396 0.2042401
 0.21062363 0.20598809 0.21685881 0.21195654 0.19105072 0.19946433
 0.20529864 0.20988844 0.20600025 0.20231778 0.20622264 0.21077426
 0.2025481  0.20783683 0.19815186 0.20223073 0.20499025 0.2055943
 0.19503121 0.20719218 0.20836595 0.20897777 0.21055694 0.20485331]
Optimal Model Parameters: {'randomforestregressor__max_features': 'sqrt', 'randomforestregressor__n_estimators': 50}


['Serialized_Predictor\\RandomForest_regressor_grid_search.serialized']

## Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier

RandomForest_classifier = RandomForestClassifier()
RandomForest_classifier_pipeline = make_pipeline(StandardScaler(), RandomForest_classifier)
RandomForest_classifier_param_grid = dict(randomforestclassifier__n_estimators=[5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
                                          randomforestclassifier__max_features = [None, "sqrt", "log2"])

RandomForest_classifier_grid_search = GridSearchCV(estimator = RandomForest_classifier_pipeline, 
                                                   param_grid=RandomForest_classifier_param_grid, 
                                                   cv = cross_validation_shufflesplit, 
                                                   n_jobs = -1,
                                                   scoring = "accuracy")

RandomForest_classifier_grid_search.fit(X_gof, y_gof)
means = RandomForest_classifier_grid_search.cv_results_['mean_test_score']
stds = RandomForest_classifier_grid_search.cv_results_['std_test_score']

print("Mean Test Score:", means)
print("Standard Deviation of Test Score:",  stds)
print('Optimal Model Parameters: ' + str(RandomForest_classifier_grid_search.best_params_))

### Serialize the gridsearchcv result

joblib.dump(RandomForest_classifier_grid_search, "RandomForest_classifier_grid_search.serialized")

Mean Test Score: [0.62667763 0.6387245  0.64682178 0.65073765 0.65382735 0.65749452
 0.65734699 0.65832069 0.65962738 0.66022593 0.63195077 0.64360985
 0.6529506  0.65831647 0.66016692 0.66304165 0.6651998  0.66645591
 0.66627466 0.6673453  0.63017619 0.64386276 0.65319929 0.65866633
 0.66232929 0.66338307 0.66393526 0.66648963 0.66589951 0.66784269]
Standard Deviation of Test Score: [0.00273082 0.00304066 0.0019767  0.00289273 0.0035491  0.00338747
 0.00363645 0.00335463 0.00496203 0.00311292 0.00237752 0.00241958
 0.00290185 0.00434374 0.00359242 0.00280806 0.00271029 0.00301458
 0.00288027 0.00352409 0.00276714 0.0033035  0.00223373 0.00315797
 0.00286713 0.00309154 0.003685   0.00429298 0.00420543 0.00267171]
Optimal Model Parameters: {'randomforestclassifier__max_features': 'log2', 'randomforestclassifier__n_estimators': 50}


['RandomForest_classifier_grid_search.serialized']

## Extremely Randomized Trees Regression

In [4]:
from sklearn.ensemble import ExtraTreesRegressor

ExtraTrees_regressor = ExtraTreesRegressor()
ExtraTrees_regressor_pipeline = make_pipeline(StandardScaler(), ExtraTrees_regressor)
ExtraTrees_regressor_param_grid = dict(extratreesregressor__n_estimators = [5, 10, 15, 20, 25, 30],
                                       extratreesregressor__max_features = [None, "sqrt", "log2"])

ExtraTrees_regressor_grid_search = GridSearchCV(estimator = ExtraTrees_regressor_pipeline, 
                                                param_grid = ExtraTrees_regressor_param_grid, 
                                                cv = cross_validation_shufflesplit, 
                                                n_jobs = -1,
                                                scoring = "neg_mean_squared_error")
ExtraTrees_regressor_grid_search.fit(X_ret, y_ret)

means = ExtraTrees_regressor_grid_search.cv_results_['mean_test_score']
stds = ExtraTrees_regressor_grid_search.cv_results_['std_test_score']

print("Mean Test Score:", means)
print("Standard Deviation of Test Score:", stds)
print('Optimal Model Parameters: ' + str(ExtraTrees_regressor_grid_search.best_params_))

### Serialize the gridsearchcv result

#joblib.dump(ExtraTrees_regressor_grid_search, "Serialized_Predictor\\ExtraTrees_regressor_grid_search.serialized")

Mean Test Score: [-0.66219396 -0.57624465 -0.5612669  -0.55953773 -0.55537434 -0.54852095
 -0.58600397 -0.55168358 -0.54158346 -0.53880377 -0.5181611  -0.52246004
 -0.59198354 -0.54847473 -0.53544909 -0.53315849 -0.52832419 -0.52606857]
Standard Deviation of Test Score: [0.19313088 0.20258063 0.20300839 0.19922378 0.19603551 0.18886411
 0.19153906 0.19031616 0.19893882 0.19998133 0.19292543 0.19582835
 0.19845686 0.2036119  0.1911792  0.19848877 0.19615517 0.19406705]
Optimal Model Parameters: {'extratreesregressor__max_features': 'sqrt', 'extratreesregressor__n_estimators': 25}


## Extremely Randomized Trees Classification

In [7]:
from sklearn.ensemble import ExtraTreesClassifier

ExtraTrees_classifier = ExtraTreesClassifier()
ExtraTrees_classifier_pipeline = make_pipeline(StandardScaler(), ExtraTrees_classifier)
ExtraTrees_classifier_param_grid = dict(extratreesclassifier__n_estimators = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
                                        extratreesclassifier__max_features = [None, "sqrt", "log2"])

ExtraTrees_classifier_grid_search = GridSearchCV(estimator = ExtraTrees_classifier_pipeline, 
                                                 param_grid=ExtraTrees_classifier_param_grid, 
                                                 cv = cross_validation_shufflesplit, 
                                                 n_jobs = -1,
                                                 scoring = "accuracy")

ExtraTrees_classifier_grid_search.fit(X_gof, y_gof)
means = ExtraTrees_classifier_grid_search.cv_results_['mean_test_score']
stds = ExtraTrees_classifier_grid_search.cv_results_['std_test_score']

print("Mean Test Score:", means)
print("Standard Deviation of Test Score:",  stds)
print('Optimal Model Parameters: ' + str(ExtraTrees_classifier_grid_search.best_params_))

### Serialize the gridsearchcv result

joblib.dump(ExtraTrees_classifier_grid_search, "Serialized_Predictor\\ExtraTrees_classifier_grid_search.serialized")

Mean Test Score: [0.6265596  0.6336579  0.64023773 0.64188164 0.64538442 0.6452664
 0.64658152 0.64621902 0.64752993 0.64708734 0.62440145 0.63392767
 0.64321362 0.64369415 0.64638762 0.64789664 0.64965436 0.6488029
 0.64929607 0.6502234  0.62564913 0.63494352 0.6418985  0.64511044
 0.6468808  0.64742876 0.64824229 0.64937194 0.65068285 0.64999578]
Standard Deviation of Test Score: [0.00337949 0.00205234 0.00248035 0.00272938 0.00252335 0.003816
 0.00297251 0.00159078 0.00148299 0.00223627 0.00183802 0.00323173
 0.00287207 0.00249777 0.00201124 0.00249578 0.00277536 0.00257433
 0.0017829  0.00276928 0.00276054 0.00317958 0.00338438 0.00217554
 0.00228289 0.00330797 0.00322674 0.00282566 0.00255792 0.00230443]
Optimal Model Parameters: {'extratreesclassifier__max_features': 'log2', 'extratreesclassifier__n_estimators': 45}


['Serialized_Predictor\\ExtraTrees_classifier_grid_search.serialized']

## AdaBoost Regression

In [8]:
from sklearn.ensemble import AdaBoostRegressor

AdaBoost_regressor = AdaBoostRegressor()
AdaBoost_regressor_pipeline = make_pipeline(StandardScaler(), AdaBoost_regressor)
AdaBoost_regressor_param_grid = dict(adaboostregressor__n_estimators = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

AdaBoost_regressor_grid_search = GridSearchCV(estimator = AdaBoost_regressor_pipeline,
                                              param_grid = AdaBoost_regressor_param_grid,
                                              cv = cross_validation_shufflesplit, 
                                              n_jobs = -1,
                                              scoring = "neg_mean_squared_error")

AdaBoost_regressor_grid_search.fit(X_ret, y_ret)
means = AdaBoost_regressor_grid_search.cv_results_["mean_test_score"]
stds = AdaBoost_regressor_grid_search.cv_results_["std_test_score"]

print("Mean Test Score:", means)
print("Standard Deviation of Test Score:", stds)
print('Optimal Model Parameters: ' + str(AdaBoost_regressor_grid_search.best_params_))

### Serialize the gridsearchcv result

joblib.dump(AdaBoost_regressor_grid_search, "Serialized_Predictor\\AdaBoost_regressor_grid_search.serialized")

Mean Test Score: [-0.52294818 -0.61078203 -0.77404321 -1.05640153 -1.09308311 -1.31863903
 -1.54731403 -2.33871837 -2.27562609 -2.49910786]
Standard Deviation of Test Score: [0.19960858 0.21957392 0.17406626 0.34719838 0.24566943 0.50521587
 0.63708795 1.65203443 2.24023025 2.94240207]
Optimal Model Parameters: {'adaboostregressor__n_estimators': 5}


['Serialized_Predictor\\AdaBoost_regressor_grid_search.serialized']

## AdaBoost Classifier

In [9]:
from sklearn.ensemble import AdaBoostClassifier

AdaBoost_classifier = AdaBoostClassifier()
AdaBoost_classifier_pipeline = make_pipeline(StandardScaler(), AdaBoost_classifier)
AdaBoost_classifier_param_grid = dict(adaboostclassifier__n_estimators = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

AdaBoost_classifier_grid_search = GridSearchCV(estimator = AdaBoost_classifier_pipeline,
                                               param_grid = AdaBoost_classifier_param_grid,
                                               cv = cross_validation_shufflesplit, 
                                               n_jobs = -1,
                                               scoring = "accuracy")

AdaBoost_classifier_grid_search.fit(X_gof, y_gof)
means = AdaBoost_classifier_grid_search.cv_results_["mean_test_score"]
stds = AdaBoost_classifier_grid_search.cv_results_["std_test_score"]

print("Mean Test Score:", means)
print("Standard Deviation of Test Score:", stds)
print('Optimal Model Parameters: ' + str(AdaBoost_classifier_grid_search.best_params_))

### Serialize the gridsearchcv result

joblib.dump(AdaBoost_classifier_grid_search, "Serialized_Predictor\\AdaBoost_classifier_grid_search.serialized")

Mean Test Score: [0.60455235 0.61641797 0.63003288 0.63487186 0.63855168 0.64113977
 0.64322627 0.64386276 0.64513151 0.64692295]
Standard Deviation of Test Score: [0.00190151 0.00361405 0.00298189 0.00234424 0.00271888 0.00265789
 0.00220597 0.00171158 0.00167511 0.00208904]
Optimal Model Parameters: {'adaboostclassifier__n_estimators': 50}


['Serialized_Predictor\\AdaBoost_classifier_grid_search.serialized']

## Gradient Boosted Decision Trees Regression

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

GradientBoosting_regressor = GradientBoostingRegressor(random_state=200)
GradientBoosting_regressor_pipeline = make_pipeline(StandardScaler(), GradientBoosting_regressor)
GradientBoosting_regressor_param_grid = dict(gradientboostingregressor__loss = ["ls", "huber"],
                                             gradientboostingregressor__n_estimators = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

GradientBoosting_regressor_grid_search = GridSearchCV(estimator = GradientBoosting_regressor_pipeline,
                                                      param_grid = GradientBoosting_regressor_param_grid,
                                                      cv = cross_validation_shufflesplit, 
                                                      n_jobs = -1,
                                                      scoring = "neg_mean_squared_error")

GradientBoosting_regressor_grid_search.fit(X_ret, y_ret)
means = GradientBoosting_regressor_grid_search.cv_results_["mean_test_score"]
stds = GradientBoosting_regressor_grid_search.cv_results_["std_test_score"]

print("Mean Test Score:", means)
print("Standard Deviation of Test Score:", stds)
print('Optimal Model Parameters: ' + str(GradientBoosting_regressor_grid_search.best_params_))

### Serialize the gridsearchcv result

#joblib.dump(GradientBoosting_regressor_grid_search, "GradientBoosting_regressor_grid_search_1.serialized")

Mean Test Score: [-0.48825926 -0.5059609  -0.52299907 -0.53077239 -0.53419236 -0.537702
 -0.54003133 -0.54239062 -0.54508562 -0.54706959 -0.47836142 -0.47753596
 -0.47693355 -0.47633282 -0.47586422 -0.47543487 -0.4750577  -0.47474371
 -0.47452067 -0.47429216]
Standard Deviation of Test Score: [0.16544151 0.16001484 0.16094999 0.16204346 0.16131302 0.16102703
 0.16142766 0.16113124 0.16170562 0.16141912 0.17500713 0.17504243
 0.17502484 0.17497571 0.17487934 0.17487102 0.17490552 0.17474193
 0.17476642 0.17463778]
Optimal Model Parameters: {'gradientboostingregressor__loss': 'huber', 'gradientboostingregressor__n_estimators': 50}


## Gradient Boosted Decision Trees Classification

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

GradientBoosting_classifier = GradientBoostingClassifier()
GradientBoosting_classifier_pipeline = make_pipeline(StandardScaler(), GradientBoosting_classifier)
GradientBoosting_classifier_param_grid = dict(gradientboostingclassifier__loss = ["deviance", "exponential"],
                                              gradientboostingclassifier__n_estimators = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

GradientBoosting_classifier_grid_search = GridSearchCV(estimator = GradientBoosting_classifier_pipeline,
                                                       param_grid = GradientBoosting_classifier_param_grid,
                                                       cv = cross_validation_shufflesplit, 
                                                       n_jobs = -1,
                                                       scoring = "accuracy")

GradientBoosting_classifier_grid_search.fit(X_gof, y_gof)
means = GradientBoosting_classifier_grid_search.cv_results_["mean_test_score"]
stds = GradientBoosting_classifier_grid_search.cv_results_["std_test_score"]

print("Mean Test Score:", means)
print("Standard Deviation of Test Score:", stds)
print('Optimal Model Parameters: ' + str(GradientBoosting_classifier_grid_search.best_params_))

### Serialize the gridsearchcv result

joblib.dump(GradientBoosting_classifier_grid_search, "Serialized_Predictor\\GradientBoosting_classifier_grid_search.serialized")

Mean Test Score: [0.60582532 0.61501855 0.63273057 0.64257292 0.64806104 0.65158068
 0.65433738 0.65669786 0.65806778 0.6591047  0.60585483 0.61404906
 0.63152504 0.64173411 0.64842775 0.65245743 0.65450177 0.65711094
 0.65823217 0.65914264]
Standard Deviation of Test Score: [0.00304631 0.00487443 0.00560444 0.00530791 0.0041869  0.00404119
 0.00360749 0.00389568 0.0035286  0.00309944 0.0030555  0.0042486
 0.00522338 0.00457067 0.00385827 0.00429193 0.00396744 0.00319346
 0.00308458 0.00280104]
Optimal Model Parameters: {'gradientboostingclassifier__loss': 'exponential', 'gradientboostingclassifier__n_estimators': 50}


['Serialized_Predictor\\GradientBoosting_classifier_grid_search.serialized']

In [17]:
import pandas as pd
import numpy as np 
import sklearn
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV

### Read Test Dataset
df_Stock_test = pd.read_csv("Desktop/finalproject_training.csv")
comp_list_test = df_Stock_test.comp_id.unique()

### Construct self-defined feature

df_Stock_test["h/l"] = df_Stock_test["m_high_adj"] / df_Stock_test["m_low_adj"] - 1
df_Stock_test["d/p"] = df_Stock_test["m_divs"] / df_Stock_test["close_adj"]
df_Stock_test["log_m_volume_adj"] = np.log(df_Stock_test["m_volume_adj"] + 1)
df_Stock_test["log_SP500"] = np.log(df_Stock_test["SP500WeeklyClose"] + 1)

feature_names = ["m_ret", "d/p", "h/l", "log_m_volume_adj", "log_SP500", 
                 "Bullish", "Bearish", "Bullish8WeekMovAvg", 
                 "epsfxq",  "mkvaltq", "gsector"]

### Select company based on data availiability of feature

comp_selection = comp_list_test
for feature in feature_names:
    comp_selection = comp_selection[df_Stock_test.groupby(df_Stock_test.comp_id, sort = False)[feature].apply(lambda x: not x.isnull().all())]
    df_Stock_test = df_Stock_test[df_Stock_test.comp_id.isin(comp_selection)]
    df_Stock_test[feature] = df_Stock_test.groupby(df_Stock_test.comp_id, sort = False)[feature].apply(lambda x: x.ffill().bfill()) 


### Create Dummy Variables for feature gsector

non_dummy_cols = df_Stock_test.columns
df_Stock_test = pd.get_dummies(df_Stock_test, columns = ["gsector"])
dummy_cols = list(set(df_Stock_test.columns) - set(non_dummy_cols))

feature_names = ["m_ret", "d/p", "h/l", "log_m_volume_adj", "log_SP500", 
                 "Bullish", "Bearish", "Bullish8WeekMovAvg", 
                 "epsfxq",  "mkvaltq"] + dummy_cols


### Construct the feature of test dataset

X_gof_test = df_Stock_test.loc[:, feature_names]

### Deserialized predictor

p_gof = joblib.load("Desktop/RandomForest_classifier_grid_search.serialized") 

### Prediction

m_gof = p_gof.predict(X_gof_test)



In [24]:
### Read Test Dataset
df_Stock_test = pd.read_csv("Desktop/finalproject_training.csv")
comp_list_test = df_Stock_test.comp_id.unique()

### Construct self-defined feature

df_Stock_test["h/l"] = df_Stock_test["m_high_adj"] / df_Stock_test["m_low_adj"] - 1
df_Stock_test["d/p"] = df_Stock_test["m_divs"] / df_Stock_test["close_adj"]
df_Stock_test["log_m_volume_adj"] = np.log(df_Stock_test["m_volume_adj"] + 1)
df_Stock_test["log_SP500"] = np.log(df_Stock_test["SP500WeeklyClose"] + 1)

feature_names = ["m_ret", "d/p", "h/l", "log_m_volume_adj", "log_SP500", 
                 "Bullish", "Bearish", "Bullish8WeekMovAvg", 
                 "epsfxq",  "mkvaltq", "gsector"]

### Select company based on data availiability of feature

comp_selection = comp_list_test
for feature in feature_names:
    comp_selection = comp_selection[df_Stock_test.groupby(df_Stock_test.comp_id, sort = False)[feature].apply(lambda x: not x.isnull().all())]
    df_Stock_test = df_Stock_test[df_Stock_test.comp_id.isin(comp_selection)]
    df_Stock_test[feature] = df_Stock_test.groupby(df_Stock_test.comp_id, sort = False)[feature].apply(lambda x: x.ffill().bfill()) 


### Create Dummy Variables for feature gsector

non_dummy_cols = df_Stock_test.columns
df_Stock_test = pd.get_dummies(df_Stock_test, columns = ["gsector"])
dummy_cols = list(set(df_Stock_test.columns) - set(non_dummy_cols))

feature_names = ["m_ret", "d/p", "h/l", "log_m_volume_adj", "log_SP500", 
                 "Bullish", "Bearish", "Bullish8WeekMovAvg", 
                 "epsfxq",  "mkvaltq"] + dummy_cols


### Construct the feature of test dataset

X_gof_test = df_Stock_test.loc[:, feature_names]

### Deserialized predictor

p_gof = joblib.load("Desktop/RandomForest_classifier_grid_search.serialized") 

### Prediction

m_gof_1 = p_gof.predict(X_gof_test)



In [25]:
np.unique(m_gof-m_gof_1)

array([0])